In [2]:
#Import all necessary modules

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from osgeo import gdal
import matplotlib.pyplot as plt

In [5]:
import os
import numpy as np

In [6]:
from PIL import Image

In [7]:
from torch.utils.data import DataLoader
from torchvision import transforms

In [8]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from osgeo import gdal

In [9]:

#Storing the directories in which the images are stored

wildfire_dir = "/content/drive/MyDrive/z227056_allbands_fire"
non_wildfire_dir = "/content/drive/MyDrive/z224082_allbands_nonfire"
input_shape = (128, 128, 3)  # Input image shape (adjust as needed)
batch_size = 32
epochs = 10

In [10]:
#Function to process the Landsat satellite images
def load_and_preprocess_image(file_path):
    dataset = gdal.Open(file_path)
    band_data_list = []
    for band_idx in [7, 4, 10]:  # Assuming bands 7, 4, and 10
        band = dataset.GetRasterBand(band_idx)
        band_data = band.ReadAsArray()
        band_data = (band_data - np.min(band_data)) / (np.max(band_data) - np.min(band_data))
        band_data_list.append(band_data)

    # Stack band data to form an RGB image
    composite_image = np.stack(band_data_list, axis=-1)
    composite_image = (composite_image * 255).astype(np.uint8)  # Convert to 8-bit uint

    return Image.fromarray(composite_image)

In [11]:
#Load the filepaths for all the images, and preprocess them with the function

fire_image_paths = [os.path.join(wildfire_dir, filename) for filename in os.listdir(wildfire_dir)]
non_image_paths = [os.path.join(non_wildfire_dir, filename) for filename in os.listdir(non_wildfire_dir)]

# Preprocess the images
fire_images = [load_and_preprocess_image(path) for path in fire_image_paths]
non_images = [load_and_preprocess_image(path) for path in non_image_paths]

<ipython-input-10-112a07a95d33>:8: RuntimeWarning: invalid value encountered in divide
  band_data = (band_data - np.min(band_data)) / (np.max(band_data) - np.min(band_data))


In [12]:
#Use GPU if available to speed up the process
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
#Assign labels to each image (1 if wildfire present, 0 if not) and store it as
# a single dataset, that will be used in training the classification model
class FireDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [14]:
#Creating labels for all the data: 1 for wildfire present, 0 for none
fire_labels = [1] * len(fire_images)
non_labels = [0] * len(non_images)

# Combine all images and labels (those with wildfires, and those without)
all_images = fire_images + non_images
all_labels = fire_labels + non_labels

# Shuffle the data
shuffled_indices = np.random.permutation(len(all_images))
shuffled_images = [all_images[i] for i in shuffled_indices]
shuffled_labels = [all_labels[i] for i in shuffled_indices]

#Transform the data to the required size and format to train under the ResNet model
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create the dataset
dataset = FireDataset(shuffled_images, shuffled_labels, transform=transform)


In [15]:
from torch.utils.data import random_split

#Splitting the test and train dataset in an 80:20 ratio
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size


train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [16]:
#creating the dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

In [17]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models


#Loading the ResNet-50 model
model = models.resnet50(pretrained=True)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
#Changing the last fully connected layer to suit our needs of binary classification
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=2)
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 42.7MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
#Training the model on the images

for epoch in range(epochs):
    model.train() #Setting the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    #Display loss of the model, as it progresses through training
    average_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {average_loss:.4f}")

print("Training complete")

Epoch [1/10] - Loss: 0.2019
Epoch [2/10] - Loss: 0.1201
Epoch [3/10] - Loss: 0.0549
Epoch [4/10] - Loss: 0.0713
Epoch [5/10] - Loss: 0.0733
Epoch [6/10] - Loss: 0.0461
Epoch [7/10] - Loss: 0.0687
Epoch [8/10] - Loss: 0.0494
Epoch [9/10] - Loss: 0.0416
Epoch [10/10] - Loss: 0.0618
Training complete


In [ ]:
model.eval()  # Set the model to evaluation mode
correct_predictions = 0
total_samples = 0

with torch.no_grad():  #Disable gradient calculation during testing
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, dim=1)  # Get predicted class

        correct_predictions += (predicted == labels).sum().item() #Calculate the number of correct predictions
        total_samples += labels.size(0)  #Total samples = Batch size

#Calculate the accuracy and display it
accuracy = correct_predictions / total_samples
print(f"Accuracy on test data: {accuracy:.2%}")

Accuracy on test data: 100.00%


In [ ]:
#Function to save the model
def save_model(model, epoch, filename):
    state = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(state, filename)

In [ ]:
file_name_model = "RESNET-50_DETECTINGWILDFIRE"
save_model(model, epoch, file_name_model)

In [18]:
#Load the model
model.load_state_dict(torch.load("/content/drive/MyDrive/SAVED_MODELS/RESNET-50_DETECTINGWILDFIRE")["model_state_dict"])

<All keys matched successfully>

In [21]:
#Testing the model on the entire dataset
data_loader = DataLoader(dataset, batch_size=len(dataset), shuffle=True, num_workers=2, pin_memory=True)


In [22]:
model.eval()  # Set the model to evaluation mode
correct_predictions = 0
total_samples = 0

with torch.no_grad():  #Disable gradient calculation during testing
    for images, labels in data_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, dim=1)  # Get predicted class

        correct_predictions += (predicted == labels).sum().item() #Calculate the number of correct predictions
        total_samples += labels.size(0)  #Total samples = Batch size

#Calculate the accuracy and display it
accuracy = correct_predictions / total_samples
print(f"Accuracy on all data: {accuracy:.2%}")

Accuracy on all data: 100.00%


In [ ]:
#Code to determine the size of the wildfire

In [ ]:
import numpy as np

lower_red_range = np.array([0.36, 0, 0])  # Example lower range for red/orange
upper_red_range = np.array([100, 0.3, 0.4])  # Example upper range for red/orange

first_image = fire_images[1]

red_channel = first_image[:, :, 0]

# Create a mask for red/orange pixels
red_orange_mask = np.all(np.logical_and(first_image >= lower_red_range, first_image <= upper_red_range), axis=-1)

num_red_orange_pixels = np.count_nonzero(red_orange_mask)

print("Number of red/orange pixels in the first image:", num_red_orange_pixels)
print("Area of fire:", num_red_orange_pixels*30," metres")


red_orange_region = np.zeros_like(first_image)
red_orange_region[red_orange_mask] = first_image[red_orange_mask]


# Count the number of red/orange pixels (True values in the mask)
num_red_orange_pixels = np.count_nonzero(red_orange_mask)

# Display the original image and the red/orange region using matplotlib
plt.figure(figsize=(10, 5))

# Original Image
plt.subplot(1, 2, 1)
plt.imshow(first_image)
plt.title("Original Image")

# Red/Orange Region
plt.subplot(1, 2, 2)
plt.imshow(red_orange_region)
plt.title(f"Red/Orange Region\nPixel Count: {num_red_orange_pixels}")

plt.tight_layout()
plt.show()